In [29]:
import pandas as pd
import freeman as fm
import networkx as nx

In [71]:
df = pd.read_csv(
    "csv/ufc-master.csv",
    usecols=(""R_fighter", "B_fighter"", "gender", "date", "Winner", "weight_class"),
)
df = df.loc[df["gender"] == "MALE"]
df.loc[:, "date"] = pd.to_datetime(df["date"], format="%m/%d/%Y", errors="ignore")
# df = df[df["date"] >= "01/01/2015"]
# df = df[
#     (df["weight_class"] == "Heavyweight")
#     | (df["weight_class"] == "Middleweight")
#     | (df["weight_class"] == "Lightweight")
# ]

# 'Middleweight', 'Featherweight', 'Heavyweight', 'Lightweight',
# 'Bantamweight', 'Welterweight', 'Light Heavyweight',
# 'Catch Weight', 'Flyweight'


SyntaxError: invalid syntax (<ipython-input-71-33684a4a27d5>, line 3)

In [72]:
df["weight_class"].unique()

array(['Middleweight', 'Featherweight', 'Heavyweight', 'Lightweight',
       'Bantamweight', 'Welterweight', 'Light Heavyweight',
       'Catch Weight', 'Flyweight'], dtype=object)

In [73]:
def unique_fights(df):
    for date in df["date"].unique():
        matches = df.loc[df["date"] == date]
        for idx, val in matches.iterrows():
            rb = matches.loc[
                (matches["R_fighter"] == val["R_fighter"])
                & (matches["B_fighter"] == val["B_fighter"])
            ]
            if rb.shape[0] != 1:
                print(rb)
                return False
            br = matches.loc[
                (matches["R_fighter"] == val["B_fighter"])
                & (matches["B_fighter"] == val["R_fighter"])
            ]
            if br.shape[0] != 0:
                print(br)
                return False
    return True

In [74]:
# Check if fights are unique
# unique_fights(df)

In [75]:
fighters = frozenset(zip(df["R_fighter"], df["B_fighter"]))

In [76]:
all_fighters = pd.DataFrame(
    pd.concat([df["R_fighter"], df["B_fighter"]]).unique(), columns=("name",)
)


In [77]:
with open("ufc.gml", "w+") as f:
    f.write("graph [\n    directed 1\n")
    for idx, fighter in all_fighters.iterrows():
        f.write("    node [\n")
        f.write(f"        id {idx}\n")
        f.write(f"        label \"{fighter['name']}\"\n")
        f.write("    ]\n")
    for fr, fb in fighters:
        f.write("    edge [\n")
        f.write(f"        source {all_fighters.index[all_fighters['name'] == fr][0]}\n")
        f.write(f"        target {all_fighters.index[all_fighters['name'] == fb][0]}\n")
        f.write("    ]\n")

    f.write("]\n")

In [78]:
g = fm.load('ufc.gml')

In [79]:
g.set_all_nodes(size=10, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.25))
g.move("kamada_kawai")
g.draw()

In [80]:
df["date"]

0      2020-10-31
1      2020-10-31
2      2020-10-31
3      2020-10-31
4      2020-10-31
          ...    
4449   2010-03-21
4450   2010-03-21
4451   2010-03-21
4452   2010-03-21
4453   2010-03-21
Name: date, Length: 4023, dtype: datetime64[ns]